
# Option Viz — Comprehensive Module Tests & Visual Demos

This notebook is a **visual, end‑to‑end testbed** for your project’s core modules.  
It prioritizes **data processing correctness** and then exercises the **volatility** and **density** stack with many **plotted use cases**.

**How to run**
1. Place this notebook alongside your modules (or run here in this environment).
2. Run top→bottom. API tests are OFF — all scenarios are synthetic and deterministic.
3. Each section explains what’s being validated.

**Contents**
1. Environment bootstrap (maps uploaded files → `src.*` packages)  
2. Data schema & loaders (round‑trip)  
3. Midprice & quote‑quality diagnostics (plots)  
4. Put–Call Parity checks (scalar & tabular)  
5. Forward & moneyness helpers  
6. No‑arbitrage diagnostics (butterfly) — visual flags  
7. SVI calibration — multiple smile regimes (flat, skewed, noisy) + plots  
8. IV surface sketch across maturities + calendar spot checks  
9. BL density extraction — different shapes; PDF/CDF; moments  
10. Appendix: Risk‑free conversions & small demos


## 1) Environment bootstrap — load your modules as `src.*` packages

In [ ]:
import sys, types, importlib.util
from pathlib import Path

BASE = Path("C:\\Users\\drewv\\Documents\\option-density-viz\\src")


def ensure_pkg(name: str):
    if name in sys.modules:
        return sys.modules[name]
    mod = types.ModuleType(name)
    mod.__package__ = name
    sys.modules[name] = mod
    return mod


# Ensure package scaffolding exists
for pkg in ["src", "src.data", "src.preprocess", "src.vol", "src.density"]:
    ensure_pkg(pkg)


def load_as(name: str, filepath: Path):
    spec = importlib.util.spec_from_file_location(name, filepath)
    mod = importlib.util.module_from_spec(spec)
    sys.modules[name] = mod
    assert spec and spec.loader, f"Cannot load spec for {name}"
    spec.loader.exec_module(mod)
    return mod


mapping = {
    # data
    "base.py": "src.data.base",
    "registry.py": "src.data.registry",
    "yf_fetcher.py": "src.data.yf_fetcher",
    "polygon_fetcher.py": "src.data.polygon_fetcher",
    "okx_fetcher.py": "src.data.okx_fetcher",
    "cache.py": "src.data.cache",
    "rate_limit.py": "src.data.rate_limit",
    "historical_loader.py": "src.data.historical_loader",
    "risk_free.py": "src.data.risk_free",
    "risk_free_fetchers.py": "src.data.risk_free_fetchers",
    # preprocess
    "forward.py": "src.preprocess.forward",
    "midprice.py": "src.preprocess.midprice",
    "pcp.py": "src.preprocess.pcp",
    # vol
    "no_arb.py": "src.vol.no_arb",
    "svi.py": "src.vol.svi",
    "surface.py": "src.vol.surface",
    # density
    "bl.py": "src.density.bl",
    "cdf.py": "src.density.cdf",
}
loaded = {}
for fname, modname in mapping.items():
    try:
        loaded[modname] = load_as(modname, BASE / fname)
        print(f"OK  | {modname} <- {fname}")
    except Exception as e:
        print(f"FAIL| {modname}: {e}")
print(f"Loaded {len(loaded)} modules.")

## 2) Core imports & configuration

In [ ]:
import math, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timezone

# Data core
from src.data.base import OptionQuote, OptionChain
from src.data.historical_loader import chain_to_dataframe, dataframe_to_chain

# Processing
from src.preprocess.midprice import add_midprice_columns
from src.preprocess.pcp import pcp_residual
from src.preprocess.forward import yearfrac, forward_price, log_moneyness

# Vol
from src.vol.no_arb import butterfly_violations
from src.vol.svi import calibrate_svi_from_quotes, svi_total_variance
from src.vol.surface import _implied_vol_black76_call  # for demos

# Density
from src.density.bl import bl_pdf_from_calls
from src.density.cdf import build_cdf, moments_from_pdf

np.set_printoptions(suppress=True, precision=6)
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 50)


def newfig(title):
    fig = plt.figure(figsize=(7.5, 4.5))
    plt.title(title)
    return fig


## 3) Data schema & loader round‑trip

We build a tiny chain, convert to DataFrame, then round‑trip back.  
This validates the **normalized schema** and the loader helpers.


In [ ]:
expiry = datetime(2026, 1, 16, tzinfo=timezone.utc)
asof = datetime(2025, 9, 8, tzinfo=timezone.utc)
quotes = [
    OptionQuote(
        "NVDA260116C00100000",
        "NVDA",
        "equity",
        expiry,
        100.0,
        "C",
        1.0,
        1.3,
        1.2,
        None,
        10,
        100,
        100,
        "USD",
        "USD",
        False,
        {},
    ),
    OptionQuote(
        "NVDA260116P00100000",
        "NVDA",
        "equity",
        expiry,
        100.0,
        "P",
        0.9,
        1.1,
        1.0,
        None,
        12,
        90,
        100,
        "USD",
        "USD",
        False,
        {},
    ),
    OptionQuote(
        "NVDA260116C00120000",
        "NVDA",
        "equity",
        expiry,
        120.0,
        "C",
        0.4,
        0.6,
        0.5,
        None,
        7,
        70,
        100,
        "USD",
        "USD",
        False,
        {},
    ),
    OptionQuote(
        "NVDA260116P00120000",
        "NVDA",
        "equity",
        expiry,
        120.0,
        "P",
        2.5,
        2.9,
        2.7,
        None,
        8,
        60,
        100,
        "USD",
        "USD",
        False,
        {},
    ),
]
chain = OptionChain("NVDA", "equity", asof, 110.0, quotes)
df = chain_to_dataframe(chain)
print("DataFrame shape:", df.shape)
display(df.head(4))
chain2 = dataframe_to_chain(df)
print(
    "Round-trip quotes:",
    len(chain2.quotes),
    " | spot:",
    chain2.spot,
    "| expiry:",
    chain2.quotes[0].expiry.date(),
)


## 4) Midprice & quote‑quality diagnostics

We compute mids, relative spreads, and flags — then **plot distributions** to audit data quality.


In [ ]:
df_quotes = pd.DataFrame(
    {
        "bid": [1.0, 2.0, None, -0.1, 0.0, 0.6, 5.5, 0.95, 2.2, 1.1],
        "ask": [1.3, None, 3.2, 0.9, 1.0, 0.8, 6.0, 1.05, 2.6, 1.4],
    }
)
out = add_midprice_columns(df_quotes, wide_rel_threshold=0.20)
display(out)

# Plot histograms for mid and rel_spread (finite only)
finite_rel = out["rel_spread"].replace([np.inf, -np.inf], np.nan).dropna()
finite_mid = out["mid"].replace([np.inf, -np.inf], np.nan).dropna()

fig = newfig("Midprice distribution")
plt.hist(finite_mid.values, bins=10)
plt.xlabel("mid")
plt.ylabel("count")
plt.show()

fig = newfig("Relative spread distribution")
plt.hist(finite_rel.values, bins=10)
plt.xlabel("rel_spread")
plt.ylabel("count")
plt.show()


## 5) Put–Call Parity (PCP) checks

We evaluate parity residuals for a few strikes and **plot residual vs strike** (positive → call‑rich).


In [ ]:
rows = []
S, r, T = 110.0, 0.04, 0.5
for K, C, P in [
    (90, 21.2, 0.2),
    (100, 12.1, 1.1),
    (110, 6.2, 3.2),
    (120, 3.0, 7.8),
    (130, 1.1, 13.7),
]:
    rows.append({"K": K, "res": pcp_residual(S, K, r, T, C, P, 0.0)})
pcp_df = pd.DataFrame(rows)
display(pcp_df)

fig = newfig("PCP residual by strike")
plt.bar(pcp_df["K"].values, pcp_df["res"].values, width=3.0)
plt.axhline(0, linestyle="--")
plt.xlabel("Strike K")
plt.ylabel("Residual  (C + K e^{-rT} - (P + S e^{-qT}))")
plt.show()


## 6) Forward & log‑moneyness helpers

We compute year‑fraction, forward from carry, and log‑moneyness across a grid.  
We **plot k vs K** to inspect the mapping.


In [ ]:
yf = yearfrac("2025-01-01", "2025-07-01")
F = forward_price(S=110.0, r=0.04, T=yf, q=0.0)
K = np.linspace(70, 150, 33)
k = log_moneyness(K, F)
print(f"yearfrac≈{yf:.4f}, forward≈{F:.4f}")

fig = newfig("Log-moneyness vs Strike")
plt.plot(K, k)
plt.xlabel("Strike K")
plt.ylabel("k = ln(K/F)")
plt.show()


## 7) No‑arbitrage (butterfly) — visualize convexity

We create a slightly **non‑convex** call curve and highlight interior points that break convexity.


In [ ]:
K = np.array([80, 90, 100, 110, 120, 130], dtype=float)
C = np.array([32.0, 24.0, 18.5, 14.8, 12.0, 10.5], dtype=float)
viol = butterfly_violations(K, C, tol=0.0)
print("Butterfly violations:", viol)

fig = newfig("Call price vs Strike (mark potential convexity breaks)")
plt.plot(K, C, marker="o")
plt.xlabel("K")
plt.ylabel("C(K)")
# Mark interior points with negative curvature if available from diagnostic
idxs = viol.get("indices", [])
if idxs:
    for i in idxs:
        plt.scatter([K[i]], [C[i]], s=80)
plt.show()


## 8) SVI calibration — multiple smile regimes

We generate three different smiles (flat, skewed, noisy) at the **same maturity T**.  
For each, we **fit SVI** in total variance space and **plot observed vs fitted** `w(k)`.


In [ ]:
def demo_svi_fit(title, F=100.0, T=0.5, regime="flat"):
    K = np.linspace(60, 140, 25)
    k = log_moneyness(K, F)
    if regime == "flat":
        iv = np.full_like(K, 0.30)
    elif regime == "skewed":
        iv = 0.25 + 0.10 * (K / F - 1.0)  # increasing to the right
        iv = np.clip(iv, 0.16, 0.50)
    elif regime == "noisy":
        rng = np.random.default_rng(123)
        base = 0.28 + 0.06 * np.tanh((K - 100) / 20.0)
        iv = np.clip(base + rng.normal(0, 0.01, size=K.size), 0.15, 0.60)
    else:
        raise ValueError("regime must be flat/skewed/noisy")
    w_obs = iv**2 * T
    fit = calibrate_svi_from_quotes(k=k, w=w_obs, T=T)
    a, b, rho, m, sigma = fit.params
    w_fit = svi_total_variance(k, a, b, rho, m, sigma)

    fig = newfig(f"SVI fit — {title}")
    plt.scatter(k, w_obs, label="observed w", s=20)
    plt.plot(k, w_fit, label="SVI fit")
    plt.xlabel("k = ln(K/F)")
    plt.ylabel("total variance w")
    plt.legend()
    plt.show()
    return fit


fit_flat = demo_svi_fit("flat smile", regime="flat")
fit_skewed = demo_svi_fit("skewed smile", regime="skewed")
fit_noisy = demo_svi_fit("noisy smile", regime="noisy")
print("Params (flat):  ", fit_flat.params)
print("Params (skewed):", fit_skewed.params)
print("Params (noisy): ", fit_noisy.params)


## 9) Sketching an IV surface across maturities

We build smiles for **T ∈ {0.25, 0.5, 1.0}**, fit SVI per expiry, and **plot w(k, T)** curves together.  
This helps visually spot **calendar issues** (w should be non‑decreasing in T for fixed k).


In [ ]:
Ts = [0.25, 0.50, 1.00]
F = 100.0
K = np.linspace(70, 130, 25)
k = log_moneyness(K, F)

w_fits = []
for T in Ts:
    # Generate a smooth smile that broadens with T
    base = 0.28 + 0.05 * np.tanh((K - 100) / 20.0)
    iv = np.clip(base * (1.0 + 0.15 * (T - 0.25)), 0.12, 0.75)
    w_obs = iv**2 * T
    fit = calibrate_svi_from_quotes(k=k, w=w_obs, T=T)
    w_fit = svi_total_variance(k, *fit.params)
    w_fits.append((T, w_fit))

fig = newfig("SVI total variance across maturities")
for T, w_fit in w_fits:
    plt.plot(k, w_fit, label=f"T={T}")
plt.xlabel("k")
plt.ylabel("w(k,T)")
plt.legend()
plt.show()


## 10) Breeden–Litzenberger (BL) density — multiple shapes

We recover risk‑neutral densities from **smoothed call curves** for three regimes:
- **Lognormal‑like** (peaked near F)  
- **Heavy‑tailed** (flatter wings)  
- **Arbitrage‑tinged** (subtle convexity issues)

For each, we **plot PDF & CDF** and compute basic **moments**.


In [ ]:
def make_call_curve(regime, F=100.0):
    K = np.linspace(60, 140, 161)
    if regime == "lognormal":
        C = np.maximum(F - K, 0.0) * 0.6 + 8.0 * np.exp(
            -((K - F) ** 2) / (2 * 20.0)
        )
    elif regime == "heavy":
        C = np.maximum(F - K, 0.0) * 0.55 + 10.0 / np.sqrt(
            1.0 + ((K - F) / 25.0) ** 2
        )
    elif regime == "arbitragey":
        base = np.maximum(F - K, 0.0) * 0.58 + 6.5 * np.exp(
            -((K - F) ** 2) / (2 * 28.0)
        )
        # introduce slight local non-convexity
        wobble = 0.4 * np.sin((K - 100) / 4.0)
        C = np.maximum(base + wobble, 0.0)
    else:
        raise ValueError("unknown regime")
    return K, C


def bl_demo(regime, T=0.5, r=0.02):
    K, C = make_call_curve(regime)
    K_grid, pdf = bl_pdf_from_calls(
        K, C, T=T, r=r, grid_size=401, return_diag=False
    )
    Kg, cdf = build_cdf(K_grid, np.maximum(pdf, 0.0))
    mom = moments_from_pdf(Kg, np.maximum(pdf, 0.0))

    fig = newfig(f"Recovered PDF — {regime}")
    plt.plot(K_grid, np.maximum(pdf, 0.0))
    plt.xlabel("K")
    plt.ylabel("pdf(K)")
    plt.show()

    fig = newfig(f"CDF — {regime}")
    plt.plot(Kg, cdf)
    plt.xlabel("K")
    plt.ylabel("CDF(K)")
    plt.show()

    print(f"{regime} moments:", mom)


for regime in ["lognormal", "heavy", "arbitragey"]:
    bl_demo(regime)


## 11) Appendix — small risk‑free conversions demo

Quick sanity checks for **continuous vs. APY vs. simple** conversions.


In [ ]:
from src.data.risk_free import (
    cont_to_apy,
    apy_to_cont,
    cont_to_simple,
    simple_to_cont,
)

r_cont = 0.045
T = 0.5
apy = cont_to_apy(r_cont)
r_back = apy_to_cont(apy)
simple = cont_to_simple(r_cont, T)
cont2 = simple_to_cont(simple, T)
print(f"cont={r_cont:.6f} -> apy={apy:.6f} -> cont_back={r_back:.6f}")
print(
    f"cont={r_cont:.6f} -> simple(T={T})={simple:.6f} -> cont_back={cont2:.6f}"
)